In [4]:
import json

# Load the Yosys-generated JSON netlist
with open('synthesized_Multiplier_4x4.json', 'r') as f:
    netlist = json.load(f)

# Specify the top module manually (adjust the name as needed)
top_module_name = 'Multiplier_4x4'
if top_module_name not in netlist.get('modules', {}):
    raise KeyError(f"Module {top_module_name} not found in the netlist.")

module = netlist['modules'][top_module_name]

# Build a mapping for port nodes where numbering restarts for each port.
# For each port, we assign new names as f"{port_name}{index}" where index restarts at 0.
port_node_mapping = {}
for port_name, port in module.get("ports", {}).items():
    bits = port.get("bits", [])
    for idx, bit in enumerate(bits):
        new_node_name = f"{port_name}{idx}"  # numbering restarts for each port
        port_node_mapping[str(bit)] = new_node_name

print("Port node mapping:", port_node_mapping)

# Prepare a circuit dictionary with a set for nodes and a list for gates.
circuit = {"nodes": set(), "gates": []}

# Define a mapping from Yosys cell names to the gate types you want.
cell_mapping = {
    "$_AND_": "AND",
    "$_OR_": "OR",
    "$_NOT_": "NOT",
    "$_ANDNOT_": "ANDNOT",
    "$_ORNOT_": "ORNOT",
    "$_NAND_": "NAND",
    "$_NOR_": "NOR",
    "$_XOR_": "XOR",
    "$_XNOR_": "XNOR",
    # Add additional mappings if desired.
}

# Iterate over each cell in the top module.
for cell_name, cell in module.get("cells", {}).items():
    yosys_type = cell["type"]
    gate_type = cell_mapping.get(yosys_type, yosys_type)
    
    gate_nodes = []
    for port, nets in cell.get("connections", {}).items():
        # In Yosys JSON, nets is usually a list (even for a single connection)
        if isinstance(nets, list):
            net = nets[0]
        else:
            net = nets
        net_str = str(net)
        # Use the new name from our port mapping if it exists, otherwise use the raw net.
        node_name = port_node_mapping.get(net_str, net_str)
        gate_nodes.append(node_name)
        circuit["nodes"].add(node_name)
    
    circuit["gates"].append({"type": gate_type, "nodes": gate_nodes})

# Convert the nodes set to a sorted list for reproducibility.
circuit["nodes"] = sorted(list(circuit["nodes"]))

# Write the resulting circuit dictionary to a JSON file.
with open('circuit.json', 'w') as outfile:
    json.dump(circuit, outfile, indent=4)

print("Circuit has been written to circuit.json")


Port node mapping: {'2': 'a0', '3': 'a1', '4': 'a2', '5': 'a3', '6': 'b0', '7': 'b1', '8': 'b2', '9': 'b3', '10': 's0', '11': 's1', '12': 's2', '13': 's3', '14': 's4', '15': 's5', '16': 's6', '17': 's7'}
Circuit has been written to circuit.json


In [ ]:
import Scripts.CustomScripts as CS
import Scripts.generate_weight_from_logic as GWL
import json

with open("circuit.json", 'r') as infile:
    circuit = json.load(infile)
print(circuit)
# Preprocess the circuit to ensure that XOR and XNOR gates have the expected number of nodes.
for gate in circuit["gates"]:
    gate_type = gate["type"]
    # Set expected number of nodes based on gate type.
    if gate_type == "XOR":
        expected_nodes = 4  # 3 provided + 1 hidden
    elif gate_type == "XNOR":
        expected_nodes = 5  # 3 provided + 2 hidden
    else:
        continue  # Skip other gate types

    gate_index = circuit["gates"].index(gate)

    # Ensure XOR has a single hidden node appended
    if gate_type == "XOR" and len(gate["nodes"]) == 3:
        # Ensure XOR has a single hidden node appended if it doesn't already have one.
        if len(gate["nodes"]) == 3:
            hidden_node = f"hidden_XOR_{gate_index}_3"
            # Append the hidden node so that the gate becomes [A, B, C, H]
            gate["nodes"].append(hidden_node)
            # Instead of simply appending, insert the hidden node into the circuit's overall node list
            # immediately after the XOR output node.
            output_node = gate["nodes"][2]  # The output node (C)
            if output_node in circuit["nodes"]:
                idx = circuit["nodes"].index(output_node)
                # Insert the hidden node right after the output node, if it isn't already there.
                if idx + 1 >= len(circuit["nodes"]) or circuit["nodes"][idx + 1] != hidden_node:
                    circuit["nodes"].insert(idx + 1, hidden_node)
            else:
                circuit["nodes"].append(hidden_node)
    # Ensure XNOR has exactly 5 nodes and in correct order
    elif gate_type == "XNOR":
        if len(gate["nodes"]) == 3:  # Initial state: ['A', 'B', 'C']
            # Generate hidden nodes
            hidden_0 = f"hidden_XNOR_{gate_index}_H0"
            hidden_1 = f"hidden_XNOR_{gate_index}_H1"
            
            # Insert H0 at position 2 (before C)
            gate["nodes"].insert(2, hidden_0)
            # Append H1 at the end
            gate["nodes"].append(hidden_1)

            # Ensure the hidden nodes are registered in the circuit's node list
            for hidden_node in [hidden_0, hidden_1]:
                if hidden_node not in circuit["nodes"]:
                    circuit["nodes"].append(hidden_node)

print(circuit)
h_total, J_total, nodes = GWL.generate_circuit_weights(circuit,SaveCircuit=True,CircuitName="8bIntFac_v5")

{'nodes': ['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', 'a0', 'a1', 'a2', 'a3', 'b0', 'b1', 'b2', 'b3', 's0', 's1', 's2', 's3', 's4', 's5', 's6', 's7'], 'gates': [{'type': 'AND', 'nodes': ['b0', 'a0', 's0']}, {'type': 'NOT', 'nodes': ['s0', '18']}, {'type': 'AND', 'nodes': ['a0', 'b1', '19']}, {'type': 'AND', 'nodes': ['b0', 'a1', '20']}, {'type': 'AND', 'nodes': ['a1', 'b1', '21']}, {'type': 'NAND', 'nodes': ['s0', '21', '22']}, {'type': 'XOR', 'nodes': ['19', '20', 's1']}, {'type': 'NAND', 'nodes': ['a0', 'b2', '23']}, {'type': 'NAND', 'nodes': ['b0', 'a2', '24']}, {'type': 'NAND', 'nodes': ['23', '24', '25']}, {'type': 'AND', 'nodes': ['a2', 'b2', '26']}, {'type': 'NAND', 'nodes'

In [20]:
idx = 13
print(circuit["nodes"][idx])
print(nodes[idx])
print(h_total[idx])

31
31
-3.0


In [21]:
# Build a mapping from node name to its corresponding h_total bias value
# (We map only as many biases as there are nodes; extra bias entries are ignored.)
node_to_bias = {}
for idx, node in enumerate(circuit["nodes"]):
    if idx < len(h_total):
        node_to_bias[node] = h_total[idx]
    else:
        node_to_bias[node] = None  # In case there are nodes with no corresponding bias

# Iterate over the gates and display the h_total values for nodes in XOR gates
print("XOR Gate h_total values:\n")
for gate in circuit["gates"]:
    if gate["type"] == "XOR":
        print(f"XOR Gate Nodes: {gate['nodes']}")
        for node in gate["nodes"]:
            bias = node_to_bias.get(node, "Not found")
            print(f"  Node: {node:20s}  h_total: {bias}")
        print("-" * 40)

XOR Gate h_total values:

XOR Gate Nodes: ['19', '20', 's1', 'hidden_XOR_6_3']
  Node: 19                    h_total: -3.0
  Node: 20                    h_total: -2.0
  Node: s1                    h_total: -1.0
  Node: hidden_XOR_6_3        h_total: 2.0
----------------------------------------
XOR Gate Nodes: ['28', '29', 's2', 'hidden_XOR_14_3']
  Node: 28                    h_total: -4.0
  Node: 29                    h_total: -3.0
  Node: s2                    h_total: -1.0
  Node: hidden_XOR_14_3       h_total: 2.0
----------------------------------------
XOR Gate Nodes: ['31', '32', '35', 'hidden_XOR_20_3']
  Node: 31                    h_total: -3.0
  Node: 32                    h_total: -3.0
  Node: 35                    h_total: -1.0
  Node: hidden_XOR_20_3       h_total: 2.0
----------------------------------------
XOR Gate Nodes: ['35', '36', '39', 'hidden_XOR_24_3']
  Node: 35                    h_total: -1.0
  Node: 36                    h_total: -1.0
  Node: 39             

In [22]:
import numpy as np
# Create a mapping from node name to its index in the weight matrix.
node_to_index = {node: idx for idx, node in enumerate(circuit["nodes"])}

# Iterate over the gates, and for each XOR gate extract and print the corresponding submatrix of J_total.
print("XOR Gate J_total submatrices:\n")
for gate in circuit["gates"]:
    if gate["type"] == "XOR":
        # Get the indices corresponding to the nodes in this XOR gate.
        indices = []
        for node in gate["nodes"]:
            if node in node_to_index:
                indices.append(node_to_index[node])
            else:
                print(f"Warning: Node '{node}' not found in node mapping.")
        if indices:
            # Extract the submatrix corresponding to these indices.
            submatrix = J_total[np.ix_(indices, indices)]
            print(f"XOR Gate Nodes: {gate['nodes']}")
            print("J_total submatrix:")
            print(submatrix)
            print("-" * 40)

XOR Gate J_total submatrices:

XOR Gate Nodes: ['19', '20', 's1', 'hidden_XOR_6_3']
J_total submatrix:
[[ 0. -1. -1.  2.]
 [-1.  0. -1.  2.]
 [-1. -1.  0.  2.]
 [ 2.  2.  2.  0.]]
----------------------------------------
XOR Gate Nodes: ['28', '29', 's2', 'hidden_XOR_14_3']
J_total submatrix:
[[ 0. -1. -1.  2.]
 [-1.  0. -1.  2.]
 [-1. -1.  0.  2.]
 [ 2.  2.  2.  0.]]
----------------------------------------
XOR Gate Nodes: ['31', '32', '35', 'hidden_XOR_20_3']
J_total submatrix:
[[ 0. -1. -1.  2.]
 [-1.  0. -1.  2.]
 [-1. -1.  0.  2.]
 [ 2.  2.  2.  0.]]
----------------------------------------
XOR Gate Nodes: ['35', '36', '39', 'hidden_XOR_24_3']
J_total submatrix:
[[ 0. -2. -1.  2.]
 [-2.  0. -1.  2.]
 [-1. -1.  0.  2.]
 [ 2.  2.  2.  0.]]
----------------------------------------
XOR Gate Nodes: ['39', '41', '43', 'hidden_XOR_28_3']
J_total submatrix:
[[ 0. -2. -1.  2.]
 [-2.  0. -1.  2.]
 [-1. -1.  0.  2.]
 [ 2.  2.  2.  0.]]
----------------------------------------
XOR Gate Nodes:

In [6]:
node_to_gates = {}
for gate in circuit["gates"]:
    for node in gate["nodes"]:
        node_to_gates.setdefault(node, []).append(gate["type"])
for node, types in node_to_gates.items():
    print(f"{node}: {types}")


b0: ['AND', 'AND', 'NAND', 'AND']
a0: ['AND', 'AND', 'NAND', 'AND']
s0: ['AND', 'NOT', 'NAND', 'NAND']
18: ['NOT', 'AND']
b1: ['AND', 'AND', 'AND', 'AND']
19: ['AND', 'XOR']
a1: ['AND', 'AND', 'AND', 'AND']
20: ['AND', 'XOR', 'NAND']
21: ['AND', 'NAND', 'AND', 'NAND']
22: ['NAND', 'NOR']
s1: ['XOR']
hidden_XOR_6_3: ['XOR']
b2: ['NAND', 'AND', 'AND', 'AND']
23: ['NAND', 'NAND']
a2: ['NAND', 'AND', 'AND', 'AND']
24: ['NAND', 'NAND']
25: ['NAND', 'AND', 'NAND']
26: ['AND', 'NAND', 'XOR']
27: ['NAND', 'AND', 'NAND']
28: ['AND', 'XOR', 'NOR']
29: ['AND', 'XOR']
s2: ['XOR']
hidden_XOR_14_3: ['XOR']
30: ['NOR', 'AND', 'XOR']
31: ['AND', 'XOR']
a3: ['AND', 'AND', 'AND', 'AND']
32: ['AND', 'XOR']
33: ['AND', 'NAND', 'NAND', 'AND']
34: ['NAND', 'NAND']
35: ['XOR', 'NAND', 'XOR']
hidden_XOR_20_3: ['XOR']
36: ['AND', 'NOT', 'NAND', 'XOR', 'NAND']
37: ['NOT', 'AND']
38: ['NAND', 'NAND']
39: ['XOR', 'NAND', 'XOR']
hidden_XOR_24_3: ['XOR']
40: ['NAND', 'NAND']
41: ['NAND', 'NAND', 'XOR']
42: ['NAND',

In [ ]:
from Scripts.generate_weight_from_json_netlist import generate_weights_from_json_netlist

h_total, J_total, nodes = generate_weights_from_json_netlist('synthesized_Multiplier_4x4',SaveCircuit=False,CircuitName="test")

Detected top module: Multiplier_4x4
Overall bias vector (h_total) in np.array format:
np.array([ 1., -3., -2.,  1.,  1.,  3.,  3.,  4., -2.,  4., -4., -3., -2., -3., -3.,  1.,  3., -1.,  2., -1.,  1.,  3., -1.,  2.,  3.,  2.,  3., -1.,  2., -2.,  3., -1.,  2., -2.,  2., -2.,  2., -3.,  3., -1.,
  2.,  3., -1.,  2.,  3., -1.,  2.,  4., -1.,  2.,  3.,  1.,  2., -2., -2.,  3., -1.,  2.,  4.,  4., -3.,  2., -2.,  3., -1.,  2.,  3., -2.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  0., -1.,  2.,
 -1.,  2., -1.,  2., -1.,  2., -1.,  2., -1.,  2.,  2.])

Overall weight matrix (J_total) in np.array format:
np.array([[ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
   0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 